<a href="https://colab.research.google.com/github/Santhosh151999/IMDB-2024-Project-01-/blob/main/IMDB2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd


### **Scraping**

In [ ]:
driver = webdriver.Safari()
driver.get("https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31&genres=comedy")

while True:
    time.sleep(5)
    title_elements = driver.find_elements(By.CLASS_NAME, "ipc-title__text")
    rating_elements = driver.find_elements(By.CLASS_NAME, "ipc-rating-star--rating")
    voting_elements = driver.find_elements(By.CLASS_NAME, "ipc-rating-star--voteCount")
    dur_elements = driver.find_elements(By.CLASS_NAME, "ipc-metadata-list-summary-item")


    titles = []
    for element in title_elements:
        titles.append(element.text)

    ratings = []
    for element in rating_elements:
        ratings.append(element.text)

    votings = []
    for element in voting_elements:
        votings.append(element.text)

    duration = []
    for element in dur_elements:
        spans = element.find_elements(By.CLASS_NAME, "sc-4b408797-8.iurwGb.dli-title-metadata-item")
        if len(spans) > 1:
            duration.append(spans[1].text)
        else:
            duration.append("N/A")


    try:
        next_button = driver.find_element(By.CLASS_NAME,"ipc-see-more__button")
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        time.sleep(1)
        next_button.click()
    except:
        print("No more pages found, breaking loop.")
        break


min_len = min(len(titles), len(ratings), len(votings), len(duration))

movie_details = pd.DataFrame({
    "Title": titles[:min_len],
    "Genre": "Comedy",
    "Ratings": ratings[:min_len],
    "Votes": votings[:min_len],
    "Duration": duration[:min_len]
})

driver.quit()


No more pages found, breaking loop.


In [ ]:
movie_details.to_csv("Comedy2024.csv")

In [ ]:
movie_details

,Title,Genre,Ratings,Votes,Duration
0,Advanced title search,Comedy,7.6,(2.8K),1h 40m
1,1. Friendship,Comedy,7.5,(195K),2h 19m
2,2. Anora,Comedy,6.6,(151K),1h 45m
3,3. Beetlejuice Beetlejuice,Comedy,6.4,(12K),1h 42m
4,4. Fight or Flight,Comedy,7.5,(500K),2h 8m
...,...,...,...,...,...
2017,2017. Prázdniny s Brouckem,Comedy,6.1,(23),1h 31m
2018,"2018. One Day League: Dead Mother, Dead All",Comedy,6.9,(11),2h 1m
2019,2019. Plant Man,Comedy,5.1,(12),56m
2020,2020. Down and Out with Donna Detweiler: The L...,Comedy,4.6,(251),1h 25m


### **Combined all Genre files into 1 file (Merging)**


In [ ]:
import pandas as pd

action_df = pd.read_csv('Action2024.csv')
comedy_df = pd.read_csv('Comedy2024.csv')
romance_df = pd.read_csv('Romance2024.csv')
crime_df = pd.read_csv('Crime2024.csv')
horror_df = pd.read_csv('Horror2024.csv')

combined_df = pd.concat([action_df, comedy_df, romance_df, crime_df, horror_df])
combined_df = combined_df.drop_duplicates(subset=['Title'])




In [ ]:
combined_df.to_csv("combined.csv")

In [ ]:
!pip install mysql-connector-python

In [ ]:
combined_df = pd.read_csv("combined.csv")
combined_df

,Unnamed: 0.1,Unnamed: 0,Title,Genre,Ratings,Votes,Duration
0,0,0,Advanced title search,Action,6.5,(242K),2h 28m
1,1,1,1. Gladiator II,Action,6.4,(12K),1h 42m
2,2,2,2. Fight or Flight,Action,5.5,(60K),2h 7m
3,3,3,3. Kraven the Hunter,Action,6.5,(173K),2h 2m
4,4,4,4. Twisters,Action,7.5,(500K),2h 8m
...,...,...,...,...,...,...,...
5623,1182,1182,1182. Bijt,Horror,4.2,(23),1h 25m
5624,1183,1183,1183. Kabir Azabi: Insi ve Cinni,Horror,2.6,(10),1h 10m
5625,1184,1184,1184. El Kafirun,Horror,4.3,(76),NaN
5626,1185,1185,1185. All of it Ends,Horror,2.6,(15),NaN


### Clearing Data (duration into mins, votings counts)

In [ ]:
import pandas as pd

def convert_duration(duration):
    if pd.isna(duration) or duration == '':
        return 0
    duration = duration.strip().lower()
    hours = 0
    minutes = 0

    if 'h' in duration:
        parts = duration.split('h')
        try:
            hours = int(parts[0].strip())
        except:
            hours = 0
        if len(parts) > 1 and 'm' in parts[1]:
            try:
                minutes = int(parts[1].replace('m', '').strip())
            except:
                minutes = 0
    elif 'm' in duration:
        try:
            minutes = int(duration.replace('m', '').strip())
        except:
            minutes = 0
    else:
        # If only a number (assume minutes)
        try:
            minutes = int(duration)
        except:
            minutes = 0

    return hours * 60 + minutes


# Load your original CSV
df = pd.read_csv("combined.csv")

# Convert Duration column
df['Duration'] = df['Duration'].apply(convert_duration)

# Save cleaned duration CSV
df.to_csv("cleaned_duration_movies.csv", index=False)
print("Duration conversion complete and saved to cleaned_duration_movies.csv")


Duration conversion complete and saved to cleaned_duration_movies.csv


In [ ]:
import pandas as pd

def convert_votes(vote_str):
    if pd.isna(vote_str):
        return 0
    vote_str = str(vote_str).strip().lower()
    vote_str = vote_str.replace('(', '').replace(')', '')  # remove parentheses
    try:
        if vote_str.endswith('k'):
            return int(float(vote_str[:-1]) * 1000)
        elif vote_str.endswith('m'):
            return int(float(vote_str[:-1]) * 1000000)
        else:
            vote_str = vote_str.replace(',', '')
            return int(vote_str)
    except Exception as e:
        print(f"Error converting votes '{vote_str}': {e}")
        return 0

# Load CSV
df = pd.read_csv("cleaned_duration_movies.csv")

# Convert votes
df['Votes'] = df['Votes'].apply(convert_votes)

# Check result
print(df[['Votes']].head(10))

# Save to new CSV
df.to_csv("IMDB_2024.csv", index=False)
print("✅ Votes conversion complete and saved to cleaned_votes_movies.csv")


    Votes
0  242000
1   12000
2   60000
3  173000
4  500000
5  633000
6  137000
7    5900
8  123000
9  159000
✅ Votes conversion complete and saved to cleaned_votes_movies.csv


### **Database Interaction**

In [ ]:
import pandas as pd
import mysql.connector

df = pd.read_csv("IMDB_2024.csv")

con = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Dark2020@',
    database='IMDB_2024'
)
cursor = con.cursor()


create_table_query = """
CREATE TABLE IF NOT EXISTS movies2024 (
    Title VARCHAR(255),
    Genre VARCHAR(100),
    Ratings FLOAT,
    Votes INT,
    Duration INT
)
"""
cursor.execute(create_table_query)


insert_query = """
INSERT INTO movies2024 (Title, Genre, Ratings, Votes, Duration)
VALUES (%s, %s, %s, %s, %s)
"""

for _, row in df.iterrows():
    title = row['Title'] if pd.notna(row['Title']) else None
    genre = row['Genre'] if pd.notna(row['Genre']) else None
    ratings = float(row['Ratings']) if pd.notna(row['Ratings']) else None
    votes = int(row['Votes']) if pd.notna(row['Votes']) else 0
    duration = int(row['Duration']) if pd.notna(row['Duration']) else 0

    cursor.execute(insert_query, (title, genre, ratings, votes, duration))


con.commit()
cursor.close()
con.close()

print("✅ Data successfully loaded into MySQL table 'movies'.")


✅ Data successfully loaded into MySQL table 'movies'.


In [ ]:
import mysql.connector

con = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Dark2020@',
    database='IMDB_2024'
)

cursor = con.cursor()
cursor.execute("SELECT * FROM movies2024")
rows = cursor.fetchall()

for row in rows:
    print(row)

('Advanced title search', 'Action', 6.5, 242000, 148)
('1. Gladiator II', 'Action', 6.4, 12000, 102)
('2. Fight or Flight', 'Action', 5.5, 60000, 127)
('3. Kraven the Hunter', 'Action', 6.5, 173000, 122)
('4. Twisters', 'Action', 7.5, 500000, 128)
('5. Deadpool & Wolverine', 'Action', 8.5, 633000, 166)
('6. Dune: Part Two', 'Action', 6.8, 137000, 122)
('7. The Ministry of Ungentlemanly Warfare', 'Action', 6.3, 5900, 107)
('8. Freaky Tales', 'Action', 6.0, 123000, 110)
('9. Venom: The Last Dance', 'Action', 6.3, 159000, 105)
('10. The Beekeeper', 'Action', 6.9, 62000, 110)
('11. Sonic the Hedgehog 3', 'Action', 7.0, 240000, 109)
('12. Civil War', 'Action', 7.5, 288000, 148)
('13. Furiosa: A Mad Max Saga', 'Action', 7.6, 36000, 178)
('14. The Count of Monte-Cristo', 'Action', 6.8, 224000, 126)
('15. The Fall Guy', 'Action', 6.5, 169000, 119)
('16. Carry-On', 'Action', 6.2, 169000, 121)
('17. Road House', 'Action', 4.0, 102000, 116)
('18. Madame Web', 'Action', 7.6, 52000, 104)
('19. Tran